### Building your network

In [1]:
from tensorflow.keras.layers import Activation, Flatten, Dropout, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.applications import DenseNet121, ResNet50, MobileNetV2
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## 정보를 입력해주세요!

In [2]:
#데이터 위치 설정

train_dir = "/content/drive/My Drive/Plant_leaf_split/dataset_split/train"
validation_dir = "/content/drive/My Drive/Plant_leaf_split/dataset_split/validation"
test_dir = "/content/drive/My Drive/Plant_leaf_split/dataset_split/test"

- 모델 저장 위치
- patience
- epoch, steps_per_epoch, validation_steps


```
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# 1 colab 파일 당 1개의 하위폴더 안에 3개의 모델이 저장됩니다.
# 반드시 model_save폴더 안에 하위 폴더를 생성하고, 밑에 있는 주소도 그에 맞게 변경해주세요.
# 그래서 colab파일 별 모델들 구분할 수 있도록 해주세요!!
# ex) '/content/drive/My Drive/model_save/하위폴더1/custom_densenet_model.h5'
# ex) '/content/drive/My Drive/model_save/하위폴더1/custom_resnet_model.h5'
# ex) '/content/drive/My Drive/model_save/하위폴더1/custom_mobilenet_model.h5'


my_callbacks = [
    EarlyStopping(patience=patience),
    ModelCheckpoint(filepath=densenet_filepath, save_best_only=True)]

history1 = custom_densenet_model.fit_generator(train_generator,
                                              steps_per_epoch=steps_per_epoch,
                                              epochs=epochs,
                                              validation_data=validation_generator,
                                              validation_steps=validation_steps,
                                              callbacks=my_callbacks)
```



EarlyStopping [patience] 설명하는 사진: 링크 들어간 후 다운로드 클릭(https://drive.google.com/file/d/1QK1tUPtSX2SEqzjwEwPDskTAAQj9g5Sl/view?usp=sharing)
 
  [위 링크 사진 내용] EarlyStopping patience가 3일 때 종료지점 

In [3]:
densenet_filepath = '/content/drive/My Drive/model_save/하위폴더1/custom_densenet_model.h5'
resnet_filepath = '/content/drive/My Drive/model_save/하위폴더1/custom_resnet_model.h5'
mobilenet_filepath = '/content/drive/My Drive/model_save/하위폴더1/custom_mobilenet_model.h5'

In [4]:
# 위 링크보고 값 결정해주세요!
patience=10

In [27]:
#데이터 갯수
#num_train_data: 33,366장, num_validation_data: 5,629장, num_test_data: 5,813장

# batch_size
train_batch_size = 64
validation_batch_size = 64
test_batch_size = 64

# train data epochs : train data 몇 번 볼지
epochs = 20

# train data steps = num_train_data / train_batch_size
steps_per_epoch = 520

# 아까 논의한대로 시간 고려해서 설정
# validation data steps = num_validation_data / validation_batch_size
validation_steps = 87


# test data steps = num_test_data / test_batch_size
test_steps = 90

In [22]:
#optimizer 설정
#ex) RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
densenet_opt = optimizers.RMSprop()
resnet_opt = optimizers.RMSprop()
mobilenet_opt = optimizers.RMSprop()

## DenseNet121

  - Size: 33MB
  - Top-1 Accuracy: 0.750
  - Top-5 Accuracy: 0.923
  - Parameters: 8,062,504
  - Depth: 121

In [10]:
densenet_model = DenseNet121(weights='imagenet',
                            include_top=False,
                            input_shape=(224,224,3))

# 레이어를 dictionary로
layer_dict = dict([(layer.name, layer) for layer in densenet_model.layers])

# pretrained model 마지막 레이어의 아웃풋
x = layer_dict['relu'].output

x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dense(33, activation='softmax')(x)

# 인풋이랑 아웃풋 합쳐서 모델 형성해줌
custom_densenet_model = Model(densenet_model.input, x)

# len(layer_dict) = 427 _ pre_trained model의 레이어의 갯수는 총 427개이다.
# 전체레이어 freeze!
for layer in custom_densenet_model.layers[:427]:
    layer.trainable = False

29089792/29084464 [==============================] - 0s 0us/step


In [11]:
custom_densenet_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
______________________________________________________________________________________________

In [12]:
len(layer_dict)

427

## ResNet50

  - Size: 98MB
  - Top-1 Accuracy: 0.749
  - Top-5 Accuracy: 0.921
  - Parameters: 25,636,712
  - Depth: -

In [13]:
resnet_model = ResNet50(weights='imagenet',
                            include_top=False,
                            input_shape=(224,224,3))

# 레이어를 dictionary로
layer_dict = dict([(layer.name, layer) for layer in resnet_model.layers])

# pretrained model 마지막 레이어의 아웃풋
x = layer_dict['conv5_block3_out'].output

x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dense(33, activation='softmax')(x)

# 인풋이랑 아웃풋 합쳐서 모델 형성해줌
custom_resnet_model = Model(resnet_model.input, x)

# len(layer_dict) = 175 _ pre_trained model의 레이어의 갯수는 총 175개이다.
# 전체레이어 freeze!
for layer in custom_resnet_model.layers[:175]:
    layer.trainable = False

94773248/94765736 [==============================] - 1s 0us/step


In [14]:
custom_resnet_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [15]:
len(layer_dict)

175

## MobileNetV2

  - Size: 14MB
  - Top-1 Accuracy: 0.713
  - Top-5 Accuracy: 0.901
  - Parameters: 3,538,984
  - Depth: 88

In [16]:
mobilenet_model = MobileNetV2(weights='imagenet',
                            include_top=False,
                            input_shape=(224,224,3))

# 레이어를 dictionary로
layer_dict = dict([(layer.name, layer) for layer in mobilenet_model.layers])

# pretrained model 마지막 레이어의 아웃풋
x = layer_dict['out_relu'].output

x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dense(33, activation='softmax')(x)

# 인풋이랑 아웃풋 합쳐서 모델 형성해줌
custom_mobilenet_model = Model(mobilenet_model.input, x)

# len(layer_dict) = 155 _ pre_trained model의 레이어의 갯수는 총 155개이다.
# 따라서 전체레이어 freeze!
for layer in custom_mobilenet_model.layers[:155]:
    layer.trainable = False

9412608/9406464 [==============================] - 0s 0us/step


In [17]:
custom_mobilenet_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
____________________________________________________________________________________________

In [18]:
len(layer_dict)

155

## Compile

In [23]:
custom_densenet_model.compile(loss='categorical_crossentropy',
                               optimizer= densenet_opt,
                               metrics=['acc'])

custom_resnet_model.compile(loss='categorical_crossentropy',
                               optimizer= resnet_opt,
                               metrics=['acc'])

custom_mobilenet_model.compile(loss='categorical_crossentropy',
                               optimizer= mobilenet_opt,
                               metrics=['acc'])

### Data preprocessing
Using ImageDataGenerator to read images from directories

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [28]:
# 노이즈 및 정규화
train_datagen = ImageDataGenerator(rescale=1./255,
                             rotation_range=40,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest')

# 평가하는 목적이기 때문에 노이즈 안됨
validation_datagen = ImageDataGenerator(rescale=1./255)



train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(224, 224), 
    batch_size=train_batch_size,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=validation_batch_size,
    class_mode='categorical')

Found 33366 images belonging to 33 classes.
Found 5629 images belonging to 33 classes.


### Fitting the model using a batch generator

In [ ]:
# densenet모델 학습

my_callbacks = [
    EarlyStopping(patience=patience),
    ModelCheckpoint(filepath=densenet_filepath, save_best_only=True)]

history1 = custom_densenet_model.fit_generator(train_generator,
                                              steps_per_epoch=steps_per_epoch,
                                              epochs=epochs,
                                              validation_data=validation_generator,
                                              validation_steps=validation_steps,
                                              callbacks=my_callbacks)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20


In [ ]:
#resnet 모델 학습

my_callbacks = [
    EarlyStopping(patience=patience),
    ModelCheckpoint(filepath=resnet_filepath, save_best_only=True)
]

history2 = custom_resnet_model.fit_generator(train_generator,
                                              steps_per_epoch=steps_per_epoch,
                                              epochs=epochs,
                                              validation_data=validation_generator,
                                              validation_steps=validation_steps,
                                              callbacks=my_callbacks)

In [ ]:
# mobilenet 모델 학습

my_callbacks = [
    EarlyStopping(patience=patience),
    ModelCheckpoint(filepath=mobilenet_filepath, save_best_only=True)
]

history3 = custom_mobilenet_model.fit_generator(train_generator,
                                              steps_per_epoch=steps_per_epoch,
                                              epochs=epochs,
                                              validation_data=validation_generator,
                                              validation_steps= validation_steps,
                                              callbacks=my_callbacks)

## Displaying curves of loss and accuracy during training

## DenseNet121

In [ ]:
acc = history1.history['acc']
val_acc = history1.history['val_acc']
loss = history1.history['loss']
val_loss = history1.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('DenseNet121 accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('DenseNet121 loss')
plt.legend()
plt.show()

#ResNet50

In [ ]:
acc = history2.history['acc']
val_acc = history2.history['val_acc']
loss = history2.history['loss']
val_loss = history2.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('ResNet50 accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('ResNet50 loss')
plt.legend()
plt.show()

#MobileNetV2

In [ ]:
acc = history3.history['acc']
val_acc = history3.history['val_acc']
loss = history3.history['loss']
val_loss = history3.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('MobileNetV2 accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('MobileNetV2 loss')
plt.legend()
plt.show()

### 일반화 성능

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(224, 224),
    batch_size=test_batch_size,
    class_mode='categorical')


In [ ]:
loaded_model1.load_model(densenet_filepath)
loaded_model2.load_model(resnet_filepath)
loaded_model3.load_model(mobilenet_filepath)

In [ ]:
score1 = loaded_model1.evaluate_generator(test_generator, steps=test_steps, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)
score2 = loaded_model2.evaluate_generator(test_generator, steps=test_steps, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)
score3 = loaded_model3.evaluate_generator(test_generator, steps=test_steps, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)

In [ ]:
print("DensNet121's acc: ",score1[1])
print("ResNet50's acc: ",score2[1])
print("MobileNetV2's acc: ",score3[1])